<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#简介" data-toc-modified-id="简介-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>简介</a></span></li><li><span><a href="#AutoModelSelection&amp;HyperparameterTuning" data-toc-modified-id="AutoModelSelection&amp;HyperparameterTuning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>AutoModelSelection&amp;HyperparameterTuning</a></span><ul class="toc-item"><li><span><a href="#Auto-HyperparameterTuning" data-toc-modified-id="Auto-HyperparameterTuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Auto HyperparameterTuning</a></span><ul class="toc-item"><li><span><a href="#算法的Intuition" data-toc-modified-id="算法的Intuition-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>算法的Intuition</a></span></li><li><span><a href="#算法实现流程" data-toc-modified-id="算法实现流程-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>算法实现流程</a></span><ul class="toc-item"><li><span><a href="#初始化" data-toc-modified-id="初始化-2.1.2.1"><span class="toc-item-num">2.1.2.1&nbsp;&nbsp;</span>初始化</a></span></li><li><span><a href="#迭代少量次" data-toc-modified-id="迭代少量次-2.1.2.2"><span class="toc-item-num">2.1.2.2&nbsp;&nbsp;</span>迭代少量次</a></span></li><li><span><a href="#参数筛选" data-toc-modified-id="参数筛选-2.1.2.3"><span class="toc-item-num">2.1.2.3&nbsp;&nbsp;</span>参数筛选</a></span></li><li><span><a href="#在剩下的N/2个参数中继续筛选" data-toc-modified-id="在剩下的N/2个参数中继续筛选-2.1.2.4"><span class="toc-item-num">2.1.2.4&nbsp;&nbsp;</span>在剩下的N/2个参数中继续筛选</a></span></li><li><span><a href="#循环直到停止" data-toc-modified-id="循环直到停止-2.1.2.5"><span class="toc-item-num">2.1.2.5&nbsp;&nbsp;</span>循环直到停止</a></span></li></ul></li></ul></li><li><span><a href="#AutoModelSelection" data-toc-modified-id="AutoModelSelection-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>AutoModelSelection</a></span></li></ul></li><li><span><a href="#结语" data-toc-modified-id="结语-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>结语</a></span></li></ul></div>

# 简介


本篇文章是接着上面一篇文章的,建议先看part1部分再看part2部分。

automl大赛有很大一部分的目的是希望lifelong的学习,在实际业务中，**每个不同的阶段都可以拿到很多新的数据,如何在每个阶段利用好新的数据同时又不浪费之前训练好的模型以及旧的数据,使我们的模型能够在最近的预测任务上达到非常好的效果？**这个问题非常有趣,应用也非常广,不过我之前没研究过此类问题,所以我此处仅仅提一下此次比赛我们所尝试的方法,当然还有很多其他fancy的方案，对这块研究感兴趣的可以自行去研究或者在公众号里留言让我也一起学习一下。

1. **方案1: 增量学习(Incremental Learning)**,和神经网络的fine tune类似,也就是用新的数据在之前训练好的模型上进行微调,既可以很好地利用旧的模型,也可以减少迭代次数,增量学习在部分数据集上表现良好;在一些数据集上表现不佳(不如直接拿最新的数据重新train一个model的效果好);
2. **方案2: 保留少数具有代表性的样本**,至于采样的策略,大家可以自己思考;(这次比赛有一种采样策略集合简单的均值集成可以提大分.....不过考虑到blind test阶段可能会控制不好内存,我们团队最后没有采用这种方案);
3. **方案3: 利用之前epoch中保存的model来进行训练**,因为有些数据集受时间的影响较小,那这个时候之前epoch训练完成的model的效果往往较好;至于如何最大化这些收益达到让模型自动选择的方案,我们团队实现了一种自动模型集成的方案。


# AutoModelSelection&HyperparameterTuning

## Auto HyperparameterTuning

自动调参的方法最常见的有**网格搜索(grid search),随机化搜索(random search),贝叶斯优化搜索(bayesian optimization search)**,试过这些方法的人普遍都能体会,这些方法往往比我们直接选择默认参数或者用经验参数的效果要好一些。但是却存在一个非常大的问题,就是这些方法**非常耗时**，一旦碰到维度高,样本个数多的数据集时候,这些方法我们几乎就可以忽略了,因为时间实在是有些长。

> 这个时候大家可以尝试一下 **胡博士之前推荐的俞洋老师们搞的ZOOpt https://github.com/eyounx/ZOOpt**


本次比赛我们并没有采用上面的方法,而是采用了**Successive Halving算法**，对这些有兴趣的可以参考下面两篇论文。

-  Non-stochastic best arm identification and hyperparameter optimization
-  Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization

此处，我仅仅从practical的角度来简单的理解该算法(我没有看过上面的论文,只是看了队友的代码了解到的这些参考文献)。

### 算法的Intuition

在本次比赛中我们采用了Successive Halving的方法,该方法的想法很简单但是却非常奏效。经常做比赛的选手如果有像我之前一样，代码运行的时候，偶尔会无聊地会默默地看着模型每迭代N轮输出中间结果的话,那么现在再回忆一下,是不是和我下面所描述的类似。


> <font color=red>如果模型受参数的影响比较大话，是不是前50轮表现好的，在后500轮的效果也往往是比较好的，我的印象中85%以上的情况是的。</font>



### 算法实现流程

为了方便大家理解,我直接用下面的图进行简单的示范。

#### 初始化
假设我们现在要调LGB模型的参数,我们希望从N个参数中选出较好的几个参数,即我们有一个容器中含有N个不同参数的模型。

![](./pic/automl_1.png)

 

#### 迭代少量次

我们对上面每个不同参数的模型迭代少量的次数(自己根据算法给的时间来估算一个量)，并计算得到每一个对应模型在验证集上面的结果（这样就会出现有的结果好,有的结果不好的情况）,示例如下：

![](./pic/automl_2.png)

 

#### 参数筛选
我们将模型在不同验证集上的表现进行排序,取表现最好的**N/2**个参数继续往下迭代,剩下的**N/2**个我们就直接舍弃


![](./pic/automl_3.png)

#### 在剩下的N/2个参数中继续筛选

- 因为LGB支持Incremental Learning,为了进一步验证模型的好坏,我们可以在之前存活下来的Model上继续迭代一定的次数,并得到对应验证集上的分数,再进行筛选

![](./pic/automl_4.png)

#### 循环直到停止

- 我们不断循环上面的流程,直到只剩下M个参数我们就停止,M可以自己定,2,3,4都可以。然后利用最终得到的M个模型设计集成的方案。




##  AutoModelSelection

其实AutoModelSelection和Auto HyperparameterTuning有很强的相关性,如果我们的自动选参的时间非常短,那也就意味着我们可以在限定的时间内对多种不同的模型进行调参,并选择最终的模型以及参数，所以最简单的就是：

> **在上面的基础之上，加上多个模型。**

![](./pic/automl_5.png)

# 结语

最后，非常感谢第四范式以及NIPS举办的此次的AutoML比赛,非常感谢队友的合作,还有群中一些大佬分享自己有趣的想法,结合上一章的内容以及这一章的内容,便可以得到此次比赛的一个Baseline了。

这一次的比赛，我个人将AutoML的内容分割成了下面传统的四块内容,有兴趣的可以参考参考。

1. AutoFeature Cleaning(例如删除缺失值过于严重的特征)
2. **AutoFeature Engineering**(上一章重点) + AutoFeature Selection(否则内存会爆炸);
3. **AutoModel Selection + AutoHyperparameter Tuning**(本章重点)
4. Concept Drifting(AutoModel Ensemble(将之前每一个epoch得到的model进行某种方式的集成) + Data augmentation(保留前面epoch具有代表性的样本) + Other Methods(多读论文))